# Upnote Markdown Manager Trial

## word splitting 

データフレームを作るところはjuliaで先に実行した。
ここではデータフレームの文書からsudachipyにて構文解析をする。

In [2]:
import polars as pl 
import sudachipy 
import os

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
os.getcwd()

'd:\\develop\\upnote_md_mgr\\notebook'

In [6]:
dfm = pl.read_csv("../data/upnote_text.csv")

In [17]:
dfm.describe()

statistic,id,fpath,update,created,category,tags,contents
str,str,str,str,str,str,str,str
"""count""","""24673""","""24673""","""24673""","""24673""","""24673""","""10466""","""24673"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""14207""","""0"""
"""mean""",null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null
"""min""","""000016e673d85d…","""- 名刺.md""","""2009-10-20T17:…","""2009-10-20T17:…","""0_業務""","""18の参加者|唄口リガチャー…",""" ""Trompeten, P…"
"""25%""",null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null
"""max""","""fff5a64a0679e1…","""ﾕcon.md""","""2024-03-06T22:…","""2024-03-06T22:…","""本""","""音響""",""" ﾕcon って何だ？ …"


HR872関連のデータのみに絞る。

In [11]:
dfm872 = dfm.filter(pl.col("tags") == "HR872")

In [12]:
dfm872

id,fpath,update,created,category,tags,contents
str,str,str,str,str,str,str
"""7d73c75ded161f…","""300Wモーター.md""","""2022-07-14T08:…","""2022-07-14T08:…","""仕事_Evernote""","""HR872""",""" 300Wモーター 金子ツ…"
"""55203fbd2731bd…","""HR 吹込管 口拡げ.md""","""2022-11-01T15:…","""2022-11-01T15:…","""仕事_Evernote""","""HR872""",""" HR 吹込管 口拡げ ど…"
"""8af8d31fb6fc3d…","""HR-672 吹込管.md""","""2022-07-26T09:…","""2022-07-26T09:…","""仕事_Evernote""","""HR872""",""" HR-672 吹込管 口…"
"""d19c050d6b9a7d…","""HR-672D ベルスクリュ…","""2022-06-13T08:…","""2022-06-13T08:…","""仕事_Evernote""","""HR872""",""" HR-672D ベルスクリ…"
"""cd3473406e90d8…","""HR-872 吹込管 2.m…","""2022-08-02T10:…","""2022-08-02T09:…","""仕事_Evernote""","""HR872""",""" HR-872 吹込管 三…"
…,…,…,…,…,…,…
"""b9a76854d102fc…","""管体+レバー支持台2 半田付…","""2022-10-28T16:…","""2022-10-28T16:…","""仕事_Evernote""","""HR872""",""" 管体+レバー支持台2 半田…"
"""cfe67cd7224278…","""管体組立 9.md""","""2022-10-26T20:…","""2022-10-26T20:…","""仕事_Evernote""","""HR872""",""" 管体組立 46班に行った…"
"""d5b76910f66938…","""設備手配表確認 2.md""","""2022-03-24T09:…","""2022-03-24T09:…","""仕事_Evernote""","""HR872""",""" 設備手配表確認 原さんか…"


In [16]:
dfm872[6,"contents"]

'\nHR-872 管体組立\n\n三宅氏の治具検図。\n\n駄目だ。まだまだだ。\n\n\n\n吹込管の入口テーパーが合ってないとか、ボルトのバランスが悪い、ブロック重そうなのにアルミでない、総形状受けやっている、\n\n668用の治具を修正するのに、昔の修正と同じ修正番号使っている、等々。\n\n\n\n\n\n\n'

sudachiによる構文解析をトライ。

In [21]:
dict = sudachipy.Dictionary()

In [23]:
tokenizer = dict.create()

In [26]:
tokens1 = tokenizer.tokenize(dfm872[6,"contents"])

In [34]:
words =  [m.surface() for m in tokens1]

for w in words:
    if len(w) > 0:
        print(w, end="|") 


|HR|-|872| |管|体|組立|

|三宅|氏|の|治具|検|図|。|

|駄目|だ|。|まだまだ|だ|。|



|吹込|管|の|入口|テーパー|が|合っ|て|ない|と|か|、|ボルト|の|バランス|が|悪い|、|ブロック|重|そう|な|の|に|アルミ|で|ない|、|総|形状|受け|やっ|て|いる|、|

|668|用|の|治具|を|修正|する|の|に|、|昔|の|修正|と|同じ|修正|番号|使っ|て|いる|、|等々|。|






|